In [88]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from dateutil import relativedelta

In [63]:
df = pd.read_csv("car_insurance_claim.csv")

##  Opis zmiennych

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10302 entries, 0 to 10301
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          10302 non-null  int64  
 1   KIDSDRIV    10302 non-null  int64  
 2   BIRTH       10302 non-null  object 
 3   AGE         10295 non-null  float64
 4   HOMEKIDS    10302 non-null  int64  
 5   YOJ         9754 non-null   float64
 6   INCOME      9732 non-null   object 
 7   PARENT1     10302 non-null  object 
 8   HOME_VAL    9727 non-null   object 
 9   MSTATUS     10302 non-null  object 
 10  GENDER      10302 non-null  object 
 11  EDUCATION   10302 non-null  object 
 12  OCCUPATION  9637 non-null   object 
 13  TRAVTIME    10302 non-null  int64  
 14  CAR_USE     10302 non-null  object 
 15  BLUEBOOK    10302 non-null  object 
 16  TIF         10302 non-null  int64  
 17  CAR_TYPE    10302 non-null  object 
 18  RED_CAR     10302 non-null  object 
 19  OLDCLAIM    10302 non-nul

In [65]:
for i in df.columns:
    print(i, df[i].dtype)
    print(df.loc[:,i].unique())

ID int64
[ 63581743 132761049 921317019 ...  67790126 849208064 680381960]
KIDSDRIV int64
[0 1 2 3 4]
BIRTH object
['16MAR39' '21JAN56' '18NOV51' ... '13AUG54' '18JUN51' '27FEB47']
AGE float64
[60. 43. 48. 35. 51. 50. 34. 54. 40. 44. 37. 53. 55. 45. 39. 59. 42. 31.
 28. 47. 46. 33. 32. 49. 36. 52. 41. 56. 38. 63. 62. 30. 29. 58. 64. 66.
 26. 61. 80. 57. nan 25. 27. 20. 21. 19. 72. 69. 24. 67. 22. 23. 65. 70.
 17. 16. 81. 73. 71. 68. 18. 76.]
HOMEKIDS int64
[0 1 2 3 4 5]
YOJ float64
[11. 10. 14. nan 12.  7.  5.  0. 13.  9. 17.  8. 15. 16.  6. 18. 23.  4.
  3.  2. 19.  1.]
INCOME object
['$67,349' '$91,449' '$52,881' ... '$164,669' '$107,204' '$53,235']
PARENT1 object
['No' 'Yes']
HOME_VAL object
['$0' '$257,252' '$124,191' ... '$332,591' '$170,611' '$197,017']
MSTATUS object
['z_No' 'Yes']
GENDER object
['M' 'z_F']
EDUCATION object
['PhD' 'z_High School' 'Bachelors' '<High School' 'Masters']
OCCUPATION object
['Professional' 'z_Blue Collar' 'Manager' 'Clerical' 'Doctor' 'Lawyer' nan
 'H

NOT OK: 

DATE:
BIRTH 

DOLLAR SIGN:
INCOME, HOME_VAL, BLUEBOOK, OLDCLAIM, CLM_AMT

CATEGORIES:
    PARENT1, CAR_USE, RED_CAR, REVOKED,
    
    'z_':    
    MSTATUS, GENDER, EDUCATION, OCCUPATION, CAR_TYPE, URBANICITY

### 2. Błędy danych

2.1. Znak $ w zmiennych ilosciowych

In [66]:
# w zmiennych z $ podmieniam znak na pusty i zamieniam na float
for i in df.columns:
    n = 0
    for j in list(df[i].values):
        if '$' in str(j):
            n += 1
            print(i)
            if n >= 1:
                df[i] = df[i].str.replace('$', '')
                df[i] = df[i].str.replace(',', '').astype(np.float32)
                break

INCOME
HOME_VAL
BLUEBOOK
OLDCLAIM
CLM_AMT


2.2. Konwersja BIRTHDATE do daty

In [67]:
# sprawdzenie czy wszystkie wartości mają dokładnie 7 znaków
df['BIRTH'].apply(lambda row: len(row)).agg(['min', 'max'])

min    7
max    7
Name: BIRTH, dtype: int64

In [68]:
# sprawdzenie czy wartości mogą pochodzić z XXI wieku
df['BIRTH'].apply(lambda row: row[-2:]).agg(['min', 'max'])
# wniosek: daty urodzenia obejmują lata 1918-1987

min    18
max    87
Name: BIRTH, dtype: object

In [69]:
df['BIRTH'] = df['BIRTH'].apply(lambda row: row[:-2] + '19' +  row[-2:])
df['BIRTH'].head()

0    16MAR1939
1    21JAN1956
2    18NOV1951
3    05MAR1964
4    05JUN1948
Name: BIRTH, dtype: object

In [70]:
df['BIRTH'] = pd.to_datetime(df['BIRTH'], format='%d%b%Y')

2.3. Kategorie zawierają przedrostek 'z_'

In [71]:
# MSTATUS, GENDER, EDUCATION, OCCUPATION, CAR_TYPE, URBANICITY
for i in ['MSTATUS', 'GENDER', 'EDUCATION', 'OCCUPATION', 'CAR_TYPE', 'URBANICITY']:
    print(df.loc[:,i].unique())

['z_No' 'Yes']
['M' 'z_F']
['PhD' 'z_High School' 'Bachelors' '<High School' 'Masters']
['Professional' 'z_Blue Collar' 'Manager' 'Clerical' 'Doctor' 'Lawyer' nan
 'Home Maker' 'Student']
['Minivan' 'Van' 'z_SUV' 'Sports Car' 'Panel Truck' 'Pickup']
['Highly Urban/ Urban' 'z_Highly Rural/ Rural']


In [72]:
for i in ['MSTATUS', 'GENDER', 'EDUCATION', 'OCCUPATION', 'CAR_TYPE', 'URBANICITY']:
    df.loc[:,i] = df.loc[:,i].apply(lambda row: row.replace('z_', '') if type(row) == str else row)

In [73]:
for i in ['MSTATUS', 'GENDER', 'EDUCATION', 'OCCUPATION', 'CAR_TYPE', 'URBANICITY']:
    print(df.loc[:,i].unique())

['No' 'Yes']
['M' 'F']
['PhD' 'High School' 'Bachelors' '<High School' 'Masters']
['Professional' 'Blue Collar' 'Manager' 'Clerical' 'Doctor' 'Lawyer' nan
 'Home Maker' 'Student']
['Minivan' 'Van' 'SUV' 'Sports Car' 'Panel Truck' 'Pickup']
['Highly Urban/ Urban' 'Highly Rural/ Rural']


Weryfikacja pozostałych kategorii

In [127]:
for i in ['PARENT1', 'CAR_USE', 'RED_CAR', 'REVOKED']:
    print(df.loc[:,i].unique())

['No', 'Yes']
Categories (2, object): ['No', 'Yes']
['Private', 'Commercial']
Categories (2, object): ['Private', 'Commercial']
['Yes' 'No']
['No', 'Yes']
Categories (2, object): ['No', 'Yes']


In [126]:
df['RED_CAR'] = df['RED_CAR'].replace(['no', 'yes'], ['No', 'Yes'])

2.4. Zmiana wartości typu 'obj' na 'category'

In [74]:
for i in df.columns:
    if df[i].dtype == 'object':
        df[i] = df[i].astype('category')

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10302 entries, 0 to 10301
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          10302 non-null  int64         
 1   KIDSDRIV    10302 non-null  int64         
 2   BIRTH       10302 non-null  datetime64[ns]
 3   AGE         10295 non-null  float64       
 4   HOMEKIDS    10302 non-null  int64         
 5   YOJ         9754 non-null   float64       
 6   INCOME      9732 non-null   float32       
 7   PARENT1     10302 non-null  category      
 8   HOME_VAL    9727 non-null   float32       
 9   MSTATUS     10302 non-null  category      
 10  GENDER      10302 non-null  category      
 11  EDUCATION   10302 non-null  category      
 12  OCCUPATION  9637 non-null   category      
 13  TRAVTIME    10302 non-null  int64         
 14  CAR_USE     10302 non-null  category      
 15  BLUEBOOK    10302 non-null  float32       
 16  TIF         10302 non-

### Walidacja dat i zmiennych ilościowych w zbiorze

Walidacja zakresu

In [80]:
df.iloc[:,1:].agg(['min', 'max'])

,KIDSDRIV,BIRTH,AGE,HOMEKIDS,YOJ,INCOME,HOME_VAL,TRAVTIME,BLUEBOOK,TIF,OLDCLAIM,CLM_FREQ,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG
min,0,1918-07-04,16.0,0,0.0,0.0,0.0,5,1500.0,1,0.0,0,0,0.0,-3.0,0
max,4,1987-12-03,81.0,5,23.0,367030.0,885282.0,142,69740.0,25,57037.0,5,13,123247.0,28.0,1


CAR_AGE -3???

In [119]:
len(df[df['CAR_AGE'] < 0])

1

In [120]:
df[df['CAR_AGE'] < 0].squeeze()

ID                      676425834
KIDSDRIV                        0
BIRTH         1952-06-02 00:00:00
AGE                            47
HOMEKIDS                        0
YOJ                            12
INCOME                      48696
PARENT1                        No
HOME_VAL                   212014
MSTATUS                        No
GENDER                          F
EDUCATION               Bachelors
OCCUPATION           Professional
TRAVTIME                       46
CAR_USE                   Private
BLUEBOOK                    15390
TIF                             4
CAR_TYPE                   Pickup
RED_CAR                        no
OLDCLAIM                    33521
CLM_FREQ                        3
REVOKED                       Yes
MVR_PTS                         1
CLM_AMT                      1469
CAR_AGE                        -3
CLAIM_FLAG                      1
URBANICITY    Highly Urban/ Urban
Name: 8771, dtype: object

Czy wszystkie dane dotyczą tego samego roku?

In [106]:
dates_check = (df[~df['AGE'].isna()]['BIRTH'] + df[~df['AGE'].isna()]['AGE'].apply(lambda row: relativedelta.relativedelta(years=row)))
dates_check.agg(['min', 'max'])

d:\Programowanie\Anaconda\lib\site-packages\pandas\core\arrays\datetimelike.py:1342: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  warnings.warn(


min   1998-11-17
max   1999-11-18
dtype: datetime64[ns]

In [112]:
dates_check.apply(lambda row: str(row.month) + '-' + str(row.year)).value_counts()

7-1999     934
3-1999     921
12-1998    907
10-1999    883
8-1999     860
1-1999     853
6-1999     841
9-1999     835
5-1999     829
2-1999     811
4-1999     806
11-1999    477
11-1998    338
dtype: int64

Czy występują braki wieku i czy można je uzupełnić wykorzystując datę urodzenia?

In [131]:
df[df['AGE'].isna()]

,ID,KIDSDRIV,BIRTH,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,...,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
295,242777133,0,1986-02-18,NaN,2,8.0,2954.0,Yes,0.0,No,...,Sports Car,Yes,10099.0,2,No,4,6142.0,5.0,1,Highly Urban/ Urban
1317,55084702,0,1983-11-21,NaN,3,0.0,0.0,Yes,0.0,No,...,Minivan,Yes,0.0,0,No,7,3092.0,8.0,1,Highly Urban/ Urban
1651,194301482,0,1984-10-18,NaN,2,0.0,0.0,Yes,0.0,No,...,SUV,Yes,0.0,0,No,0,3444.0,6.0,1,Highly Urban/ Urban
3749,919557945,0,1986-02-17,NaN,2,0.0,NaN,Yes,0.0,No,...,Sports Car,Yes,25235.0,1,Yes,0,2541.0,13.0,1,Highly Urban/ Urban
4365,968439150,0,1984-02-15,NaN,3,12.0,16227.0,No,NaN,Yes,...,Van,Yes,0.0,0,No,6,5640.0,1.0,1,Highly Urban/ Urban
4554,285745137,0,1985-08-17,NaN,0,9.0,14484.0,No,0.0,No,...,Sports Car,Yes,4518.0,1,No,1,4798.0,1.0,1,Highly Urban/ Urban
5227,945868698,0,1987-12-03,NaN,0,0.0,0.0,No,NaN,Yes,...,SUV,Yes,4656.0,4,No,3,0.0,1.0,0,Highly Rural/ Rural


In [130]:
df[df['AGE'].isna()]['BIRTH']

295    1986-02-18
1317   1983-11-21
1651   1984-10-18
3749   1986-02-17
4365   1984-02-15
4554   1985-08-17
5227   1987-12-03
Name: BIRTH, dtype: datetime64[ns]

### Analiza i imputacja braków danych